In [1]:
import django, sys, os
sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from sklearn.decomposition import NMF
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier

from scoping.models import *
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from itertools import product, combinations
from utils.text import *
from scipy.sparse import find

import math
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

from MulticoreTSNE import MulticoreTSNE as mTSNE
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import coverage_error, label_ranking_average_precision_score, label_ranking_loss
from pycountry_convert import  country_alpha2_to_continent_code, country_alpha3_to_country_alpha2
import pickle
import scipy.sparse
import datetime
import scipy.stats as st
django.db.connection.close()


In [2]:
pid = 178
qid = 8145
qid = 8272
q = Query.objects.get(pk=qid)
project = Project.objects.get(pk=pid)

print(q.doc_set.count())

379897


# Economic inequality

In [3]:
predicted_ids = pd.read_csv('../data/doctables/predicted_relevant_all_ids.csv')
pds = predicted_ids[predicted_ids['prediction']>0.1]

assigned = set(DocOwnership.objects.filter(query__project=project).values_list('doc__id',flat=True))
print(len(assigned))

2969


In [42]:
assigned = set(DocOwnership.objects.filter(query__project=project).values_list('doc__id',flat=True))
print(len(assigned))

s = "economic inequality; economic activity; GDP; gender; indigenous people; indigenous knowledge; indigenous communit"
inclusion = s.replace('; ','|')

se = "species"
exclusions = se.replace(', ','|')

p = 0.1

pds = predicted_ids[predicted_ids['prediction']>p]

dids = set([])
eids = set([])
for f in ["title__iregex","content__iregex"]:
    print(f)
    dids = dids | set(q.doc_set.filter(**{f: inclusion}).values_list('pk',flat=True))
    eids = eids | set(q.doc_set.filter(**{f: exclusions}).values_list('pk',flat=True))
    
doc_ids = dids - eids
    
docs = Doc.objects.filter(pk__in=doc_ids).exclude(id__in=assigned)

#docs = q.doc_set.filter(title__iregex=inclusion).exclude(id__in=assigned).exclude(content__iregex=exclusions)
r_docs = Doc.objects.filter(id__in=doc_ids & set(pds['id']))


docs = q.doc_set.filter(title__iregex=inclusion).exclude(id__in=assigned)#.exclude(content__iregex=exclusions)
r_docs = q.doc_set.filter(title__iregex=inclusion, id__in=pds['id']).exclude(id__in=assigned)

print(docs.count(), r_docs.count())

2545
title__iregex
content__iregex
892 97


In [43]:
for d in Doc.objects.filter(pk__in=random.sample(list(docs.values_list('pk',flat=True)),5)):
    print(d.title)

Gender and occupational perspectives on adaptation to climate extremes in the Afram Plains of Ghana
Resilience of the poorest: coping strategies and indigenous knowledge of living with the floods in Northern Namibia
Revisiting Jewson and Mason: The Politics of Gender Equality in UK Local Government in a Cold Climate
Does sex matter? Gender-specificity and its influence on site-chronologies in the common dioecious shrub Juniperus communis
Month of birth and schizophrenia in Taiwan: Effect of gender, family history and age at onset


In [44]:
for d in Doc.objects.filter(pk__in=random.sample(list(r_docs.values_list('pk',flat=True)),5)):
    print(d.title)

The persistence of shocks in GDP and the estimation of the potential economic costs of climate change
2014 PSA Presidential Address (Un)Changing Institutions: Work, Family, and Gender in the New Economy
Gender, Age and Season as Modifiers of the Effects of Diurnal Temperature Range on Emergency Room Admissions for Cause-Specific Cardiovascular Disease among the Elderly in Beijing
Hope and Worry: Gendered Emotional Geographies of Climate Change in Three Vulnerable US Communities
Urban outdoor thermal perception in hot arid Beer Sheva, Israel: Methodological and gender aspects


In [45]:
users = User.objects.filter(username__in=["inga.menke@climateanalytics.org","marina.andrijevic@climateanalytics.org"])
tag, created = Tag.objects.get_or_create(query=q,title=s+" 2 - prediction>0.1")
print(tag, created)
for i,d in enumerate(Doc.objects.filter(pk__in=random.sample(list(r_docs.values_list('pk',flat=True)),50))):
    u = users[i%2]
    d.tag.add(tag)
    do, created = DocOwnership.objects.get_or_create(
        doc=d,
        query=q,
        tag=tag,
        user=u
    ) 
tag.update_tag()      

economic inequality; economic activity; GDP; gender; indigenous people; indigenous knowledge; indigenous communit 2 - prediction>0.1 True


# Soil moisture


1943


In [46]:
assigned = set(DocOwnership.objects.filter(query__project=project).values_list('doc__id',flat=True))
print(len(assigned))

s = "Soil moisture, evapotranspiration, flood*, snow cover, groundwater, water quality"
inclusion = s.replace(', ','|')
se = "runoff, streamflow"
exclusions = se.replace(', ','|')

p = 0.5

pds = predicted_ids[predicted_ids['prediction']>p]

dids = set([])
eids = set([])
for f in ["title__iregex","content__iregex"]:
    print(f)
    dids = dids | set(q.doc_set.filter(**{f: inclusion}).values_list('pk',flat=True))
    eids = eids | set(q.doc_set.filter(**{f: exclusions}).values_list('pk',flat=True))
    
doc_ids = dids - eids
    
docs = Doc.objects.filter(pk__in=doc_ids).exclude(id__in=assigned)

#docs = q.doc_set.filter(title__iregex=inclusion).exclude(id__in=assigned).exclude(content__iregex=exclusions)
r_docs = Doc.objects.filter(id__in=doc_ids & set(pds['id']))

print(docs.count(), r_docs.count())

print('###\n all query docs\n')

for d in Doc.objects.filter(pk__in=random.sample(list(docs.values_list('pk',flat=True)),5)):
    print(d.title)
    
print(f'###\n p > {p}\n')

for d in Doc.objects.filter(pk__in=random.sample(list(r_docs.values_list('pk',flat=True)),5)):
    print(d.title)
    print(d.content)

2595
title__iregex
content__iregex
47954 723
###
 all query docs

Weather patterns in eastern Slovakia 1717-1730, based on records from the Breslau meteorological network
Spatiotemporal Characteristics of Dry-Wet Abrupt Transition Based on Precipitation in Poyang Lake Basin, China
Living with risk of natural disasters
EFFECTS OF CLIMATE CHANGE ON THE URBAN TOURIST AND COASTAL AREA OF VINA DEL MAR: SURVEY OF DAMAGE FOR FLOODING BY STORM SURGE AND PERCEPTION OF SECURITY
Assessment of the toxic potential of rainwater precipitation: First evidence from a case study in three Greek cities
###
 p > 0.5

Impacts of recent climate change on dry-land crop water consumption in the northern agro-pastoral transitional zone of China
Climate change has substantially impacted crop growth and development in the northern agro-pastoral transitional zone. Examination of the response of crop water consumption to climate change may provide a guide for adapting local agricultural production and ecological co

In [47]:
users = User.objects.filter(username__in=[
    "quentin.lejeune@climateanalytics.org",
    "chelsea.jones@climateanalytics.org",
    "agathe.lucas@climateanalytics.org",
])
tag, created = Tag.objects.get_or_create(query=q,title=s+" 2 - prediction>0.2")
print(tag, created)
for i,d in enumerate(Doc.objects.filter(pk__in=random.sample(list(r_docs.values_list('pk',flat=True)),25*3))):
    u = users[i%users.count()]
    d.tag.add(tag)
    do, created = DocOwnership.objects.get_or_create(
        doc=d,
        query=q,
        tag=tag,
        user=u
    ) 
tag.update_tag()


Soil moisture, evapotranspiration, flood*, snow cover, groundwater, water quality 2 - prediction>0.2 True


# TES

In [85]:
tag, created = Tag.objects.get_or_create(query=q,title="TES keywords predicted category relevant")
tag_dors = tag.docownership_set.values('relevant').annotate(n=Count('pk'))
print(list(tag_dors.order_by('-n')))
tag_dos = tag.docownership_set.values('doc__id')
ducs = DocUserCat.objects.filter(doc__id__in=tag_dos,category__level=4).values('category__name').annotate(n=Count('pk'))
list(ducs.order_by('-n'))

[{'relevant': 1, 'n': 21}, {'relevant': 0, 'n': 20}, {'relevant': 2, 'n': 10}, {'relevant': 3, 'n': 9}]


[{'category__name': '53 Mortality and growth', 'n': 12},
 {'category__name': '59 Wildfires', 'n': 11},
 {'category__name': '51 Distribution and range shifts (Terrestrial and freshwater)',
  'n': 8},
 {'category__name': '55 Community composition and interaction', 'n': 8},
 {'category__name': '54 Physiology and metabolism', 'n': 2},
 {'category__name': '28 River runoff', 'n': 1},
 {'category__name': '26 Drought frequency and intensity', 'n': 1},
 {'category__name': '24 Water level (lake, reservoir, groundwater)', 'n': 1}]

In [84]:
tag = Tag.objects.get(query=q,title="TES keywords prediction>0.2")
tag_dors = tag.docownership_set.values('relevant').annotate(n=Count('pk'))
print(list(tag_dors.order_by('-n')))
tag_dos = tag.docownership_set.values('doc__id')
ducs = DocUserCat.objects.filter(doc__id__in=tag_dos,category__level=4).values('category__name').annotate(n=Count('pk'))
list(ducs.order_by('-n'))

[{'relevant': 1, 'n': 20}, {'relevant': 0, 'n': 20}, {'relevant': 2, 'n': 11}, {'relevant': 3, 'n': 9}]


[{'category__name': '53 Mortality and growth', 'n': 6},
 {'category__name': '45 Ocean ecosystem productivity', 'n': 4},
 {'category__name': '55 Community composition and interaction', 'n': 4},
 {'category__name': '56 Terrestrial carbon cycle', 'n': 3},
 {'category__name': '54 Physiology and metabolism', 'n': 3},
 {'category__name': '48 Carbon cycle (marine & coastal)', 'n': 3},
 {'category__name': '02 Air or land surface temperature changes', 'n': 2},
 {'category__name': '08 Changes in strong precipitation', 'n': 2},
 {'category__name': '19 Water quality/chemistry (oceans)', 'n': 2},
 {'category__name': '22 Water quality/chemistry (freshwater)', 'n': 2},
 {'category__name': '28 River runoff', 'n': 2},
 {'category__name': '66 Health', 'n': 2},
 {'category__name': '46 Changes in kelp forests', 'n': 1},
 {'category__name': '47 Seagrass', 'n': 1},
 {'category__name': '36 Species distribution (marine & coastal)', 'n': 1},
 {'category__name': '25 Evapotranspiration', 'n': 1},
 {'category__na

In [7]:
django.db.connection.close()

assigned = set(DocOwnership.objects.filter(query__project=project).values_list('doc__id',flat=True))
print(len(assigned))

s = "CO2.{,2}concentration, carbon flux, NPP, disturbance, decomposition, fire weather, area burnt, pest outbreak, fitness, biome shift"
s+= ", water.{,2}borne, vector.{,2}borne, heat.{,2}stress, drought.{,2}stress, infestation, insect, fungh, peat, encroachment, drainage"
s+= ", stratification, algae, phytoplankton, pest, biodiversity"
inclusion = s.replace(', ','|')
se = "tree ring, phenolog, flowering, mortality, growth"
exclusions = se.replace(', ','|')

p = 0.5

pds = predicted_ids[predicted_ids['prediction']>p]
c_ids = pd.read_csv(f'../data/latest_predicted_12 - Terrestrial ES')

dids = set([])
eids = set([])
for f in ["title__iregex","content__iregex"]:
    print(f)
    dids = dids | set(q.doc_set.filter(**{f: inclusion}).values_list('pk',flat=True))
    eids = eids | set(q.doc_set.filter(**{f: exclusions}).values_list('pk',flat=True))
    
doc_ids = dids - eids
    
docs = Doc.objects.filter(pk__in=doc_ids).exclude(id__in=assigned)

#docs = q.doc_set.filter(title__iregex=inclusion).exclude(id__in=assigned).exclude(content__iregex=exclusions)
r_docs = Doc.objects.filter(id__in=doc_ids & set(pds['id']))
rc_docs = Doc.objects.filter(id__in=doc_ids & set(pds['id']) & set(c_ids['id']))

print(docs.count(), r_docs.count(), rc_docs.count())

print('###\n all query docs\n')

for d in Doc.objects.filter(pk__in=random.sample(list(docs.values_list('pk',flat=True)),5)):
    print(d.title)
    
print(f'###\n p > {p}\n')

for d in Doc.objects.filter(pk__in=random.sample(list(r_docs.values_list('pk',flat=True)),5)):
    print(d.title)
    
print(f'###\n predicted cat')

for d in Doc.objects.filter(pk__in=random.sample(list(rc_docs.values_list('pk',flat=True)),5)):
    print(d.title)

2973
title__iregex
content__iregex
59278 582 99
###
 all query docs

Holocene climate variability, vegetation dynamics and fire regime in the central Pyrenees: the Basa de la Mora sequence (NE Spain)
Comparison of "warm and wet" and "cold and icy" scenarios for early Mars in a 3-D climate model
Shift in community structure in an early-successional Mediterranean shrubland driven by long-term experimental warming and drought and natural extreme droughts
DEPOSITIONAL-ENVIRONMENTS OF THE HART COAL ZONE (PALEOCENE), WILLOW BUNCH COALFIELD, SOUTHERN SASKATCHEWAN, CANADA FROM PETROGRAPHIC, PALYNOLOGICAL, PALEOBOTANICAL, MINERAL AND TRACE-ELEMENT STUDIES
Land-use intensity alters both the source and fate of CO2 within eight sub-tropical estuaries
###
 p > 0.5

LONG-TERM POPULATION TRENDS OF COLONIAL WADING BIRDS BREEDING IN DONANA (SW SPAIN) IN RELATION TO ENVIRONMENTAL AND ANTHROPOGENIC FACTORS
Humidity-regulated dormancy onset in the Fabaceae: a conceptual model and its ecological implicatio

In [69]:
users = User.objects.filter(username__in=[
    #"burcu.yesil@climateanalytics.org",
    "robert.brecha@climateanalytics.org",
    #"gerrithansen@mailbox.org"
])
tag, created = Tag.objects.get_or_create(query=q,title="TES keywords prediction>0.2")
print(tag, created)
for i,d in enumerate(Doc.objects.filter(pk__in=random.sample(list(r_docs.values_list('pk',flat=True)),5*users.count()))):
    u = users[i%users.count()]
    d.tag.add(tag)
    do, created = DocOwnership.objects.get_or_create(
        doc=d,
        query=q,
        tag=tag,
        user=u
    ) 
tag.update_tag()

CO2.{,2}concentration, carbon flux, NPP, disturbance, decomposition, fire weather, area burnt, pest outbreak, fitness, biome shift, water.{,2}borne, vector.{,2}borne, heat.{,2}stress, drought.{,2}stress, infestation, insect, fungh, peat, encroachment, drainage, stratification, algae, phytoplankton prediction>0.2 True


In [111]:
users = User.objects.filter(username__in=[
    "burcu.yesil@climateanalytics.org",
    "robert.brecha@climateanalytics.org",
    "gerrithansen@mailbox.org"
])
tag, created = Tag.objects.get_or_create(query=q,title="TES keywords prediction>0.2")
print(tag, created)
for u in users:
    print(u)
    print(tag.docownership_set.filter(user=u).count())

TES keywords prediction>0.2 False
gerrithansen@mailbox.org
20
robert.brecha@climateanalytics.org
20
burcu.yesil@climateanalytics.org
20


In [112]:
tag, created = Tag.objects.get_or_create(query=q,title="TES keywords predicted category relevant 3")
print(tag, created)
users = User.objects.filter(username__in=[
    "robert.brecha@climateanalytics.org",
    "gerrithansen@mailbox.org"
])
bu = User.objects.get(username="burcu.yesil@climateanalytics.org")
for u in users:
    dos = tag.docownership_set.filter(user=u,relevant=0)
    print(dos.count())
    for do in dos[:10]:
        do.user=bu
        do.save()

TES keywords predicted category relevant False
27
30


In [8]:
users = User.objects.filter(username__in=[
    "robert.brecha@climateanalytics.org",
    #"gerrithansen@mailbox.org"
])
tag, created = Tag.objects.get_or_create(query=q,title="TES keywords predicted category relevant 3")
print(tag, created)
for i,d in enumerate(Doc.objects.filter(pk__in=random.sample(list(rc_docs.values_list('pk',flat=True)),5))):
    u = users[i%users.count()]
    d.tag.add(tag)
    do, created = DocOwnership.objects.get_or_create(
        doc=d,
        query=q,
        tag=tag,
        user=u
    ) 
tag.update_tag()

TES keywords predicted category relevant 3 False


# food, malnutrition, conflict..

In [53]:
tag = Tag.objects.get(query=q,title="Food, conflict, migration - prediction>0.2")
tag_dos = tag.docownership_set.values('doc__id')
tag_dors = tag.docownership_set.values('relevant').annotate(n=Count('pk'))
print(list(tag_dors.order_by('-n')))
ducs = DocUserCat.objects.filter(doc__id__in=tag_dos,category__level=4).values('category__name').annotate(n=Count('pk'))
list(ducs.order_by('-n'))

[{'relevant': 2, 'n': 22}, {'relevant': 0, 'n': 11}, {'relevant': 1, 'n': 6}, {'relevant': 3, 'n': 1}]


[{'category__name': '73 Displacement and migration', 'n': 3},
 {'category__name': '09 Atmospheric/marine circulation or teleconnections',
  'n': 1},
 {'category__name': '10 Wind speed', 'n': 1},
 {'category__name': '24 Water level (lake, reservoir, groundwater)', 'n': 1},
 {'category__name': '60 Other (Terrestrial and freshwater)', 'n': 1},
 {'category__name': '69 Crop yields', 'n': 1},
 {'category__name': '70 Food prices', 'n': 1},
 {'category__name': '07 Aridity/dryness', 'n': 1},
 {'category__name': '80 Livestock management', 'n': 1}]

In [58]:
print(predicted_ids.loc[predicted_ids['id']==d.id,"prediction"].values[0])

0.3696876377976783


In [48]:
assigned = set(DocOwnership.objects.filter(query__project=project).values_list('doc__id',flat=True))
print(len(assigned))

s = "Food price, malnutrition, conflict, human displacement, internal displacement, internally displaced, migration"
inclusion = s.replace(', ','|')
print(inclusion)
se = "species, bird, mammal, fish, ecosystem, passerine, plankton, salmon, plant migration, eels, inlet migration, alligator, conflicting, aphid, reptile"
se += ", thward migration, stward migration, horizontal migration, vegetation migration, tuna, meridional migration, vertical migration"
se += ", autumn migration, spring migration, summer migration, winter migration, meridional migration, wildlife migration, lateral migration, animal migration, ward migration, seasonal migration"
se += ", turtle, breeding, butterfly, elephant, waterfowl, inland migration"
se += ", (?:migration\w*\W+(?:\w+\W+){0,5}?SST|SST\W+(?:\w+\W+){0,5}?migration*)"
se += ", (?:migration\w*\W+(?:\w+\W+){0,5}?marsh\w*|marsh\w*\W+(?:\w+\W+){0,5}?migration*)"
se += ", (?:migration\w*\W+(?:\w+\W+){0,5}?atmospher\w*|atmospher\w*\W+(?:\w+\W+){0,5}?migration*)"
print(se)
exclusions = se.replace(', ','|')

print(exclusions)

p = 0.3

pds = predicted_ids[predicted_ids['prediction']>p]

dids = set([])
eids = set([])
for f in ["title__iregex","content__iregex"]:
    print(f)
    dids = dids | set(q.doc_set.filter(**{f: inclusion}).values_list('pk',flat=True))
    eids = eids | set(q.doc_set.filter(**{f: exclusions}).values_list('pk',flat=True))
    
doc_ids = dids - eids
    
docs = Doc.objects.filter(pk__in=doc_ids).exclude(id__in=assigned)

#docs = q.doc_set.filter(title__iregex=inclusion).exclude(id__in=assigned).exclude(content__iregex=exclusions)
r_docs = Doc.objects.filter(id__in=doc_ids & set(pds['id']))

print(docs.count(), r_docs.count())

print('###\n all query docs\n')

for d in Doc.objects.filter(pk__in=random.sample(list(docs.values_list('pk',flat=True)),5)):
    try:
        print(predicted_ids.loc[predicted_ids['id']==d.id,"prediction"].values[0])
    except:
        print(predicted_ids.loc[predicted_ids['id']==d.id,"prediction"].values)
    print(d.title)
    print()
    
print(f'###\n p > {p}\n')

for d in Doc.objects.filter(pk__in=random.sample(list(r_docs.values_list('pk',flat=True)),5)):
    try:
        print(predicted_ids.loc[predicted_ids['id']==d.id,"prediction"].values[0])
    except:
        print(predicted_ids.loc[predicted_ids['id']==d.id,"prediction"].values)
    print(d.title)
    print(d.content)
    print()
    


2912
Food price|malnutrition|conflict|human displacement|internal displacement|internally displaced|migration
species, bird, mammal, fish, ecosystem, passerine, plankton, salmon, plant migration, eels, inlet migration, alligator, conflicting, aphid, reptile, thward migration, stward migration, horizontal migration, vegetation migration, tuna, meridional migration, vertical migration, autumn migration, spring migration, summer migration, winter migration, meridional migration, wildlife migration, lateral migration, animal migration, ward migration, seasonal migration, turtle, breeding, butterfly, elephant, waterfowl, inland migration, (?:migration\w*\W+(?:\w+\W+){0,5}?SST|SST\W+(?:\w+\W+){0,5}?migration*), (?:migration\w*\W+(?:\w+\W+){0,5}?marsh\w*|marsh\w*\W+(?:\w+\W+){0,5}?migration*), (?:migration\w*\W+(?:\w+\W+){0,5}?atmospher\w*|atmospher\w*\W+(?:\w+\W+){0,5}?migration*)
species|bird|mammal|fish|ecosystem|passerine|plankton|salmon|plant migration|eels|inlet migration|alligator|conf

In [49]:
users = User.objects.filter(username__in=[
    #"nicole.vanmaanen@climateanalytics.org",
    "carl.schleussner@climateanalytics.org"
])
tag, created = Tag.objects.get_or_create(query=q,title="Food, conflict, migration - prediction>0.3 4")
print(tag, created)
for i,d in enumerate(Doc.objects.filter(pk__in=random.sample(list(r_docs.values_list('pk',flat=True)),25))):
    u = users[i%users.count()]
    d.tag.add(tag)
    do, created = DocOwnership.objects.get_or_create(
        doc=d,
        query=q,
        tag=tag,
        user=u
    ) 
tag.update_tag()

Food, conflict, migration - prediction>0.3 4 True


0

In [8]:
users = User.objects.filter(username__in=[
    #"nicole.vanmaanen@climateanalytics.org",
    "carl.schleussner@climateanalytics.org"
])
uc = users.count()
tag, created = Tag.objects.get_or_create(query=q,title="Food, conflict, migration - prediction>0.2 - 2")
print(tag, created)
for i,d in enumerate(Doc.objects.filter(pk__in=random.sample(list(r_docs.values_list('pk',flat=True)),uc*20))):
    u = users[i%uc]
    d.tag.add(tag)
    do, created = DocOwnership.objects.get_or_create(
        doc=d,
        query=q,
        tag=tag,
        user=u
    ) 
tag.update_tag()

Food, conflict, migration - prediction>0.2 - 2 False


# Infrastructure, communities, land use...


In [12]:
tag, created = Tag.objects.get_or_create(query=q,title="Infrastructure, settlements - prediction > 0.2")
tag_dos = tag.docownership_set.values('doc__id')
ducs = DocUserCat.objects.filter(doc__id__in=tag_dos,category__level=4).values('category__name').annotate(n=Count('pk'))
list(ducs.order_by('-n'))

[{'category__name': '66 Health', 'n': 2},
 {'category__name': '69 Crop yields', 'n': 2},
 {'category__name': '28 River runoff', 'n': 2},
 {'category__name': '79 Livestock health', 'n': 1},
 {'category__name': '80 Livestock management', 'n': 1},
 {'category__name': '05 Changes in precipitation', 'n': 1},
 {'category__name': 'Food, fibre, and other ecosystem products', 'n': 1},
 {'category__name': '59 Wildfires', 'n': 1},
 {'category__name': '73 Displacement and migration', 'n': 1}]

In [17]:
django.db.connection.close()
assigned = set(DocOwnership.objects.filter(query__project=project).values_list('doc__id',flat=True))
print(len(assigned))

s = "infrastructure, coastal cit*, coastal settlement, indigenous"
inclusion = s.replace(', ','|')
se = "species, bird, mammal, fish, ecosystem, animal"
exclusions = se.replace(', ','|')

p = 0.2

pds = predicted_ids[predicted_ids['prediction']>p]

dids = set([])
eids = set([])
for f in ["title__iregex","content__iregex"]:
    print(f)
    dids = dids | set(q.doc_set.filter(**{f: inclusion}).values_list('pk',flat=True))
    eids = eids | set(q.doc_set.filter(**{f: exclusions}).values_list('pk',flat=True))
    
doc_ids = dids - eids
    
docs = Doc.objects.filter(pk__in=doc_ids).exclude(id__in=assigned)

#docs = q.doc_set.filter(title__iregex=inclusion).exclude(id__in=assigned).exclude(content__iregex=exclusions)
r_docs = Doc.objects.filter(id__in=doc_ids & set(pds['id']))

print(docs.count(), r_docs.count())

print('###\n all query docs\n')

for d in Doc.objects.filter(pk__in=random.sample(list(docs.values_list('pk',flat=True)),5)):
    print(d.title)
    
print(f'###\n p > {p}\n')

for d in Doc.objects.filter(pk__in=random.sample(list(r_docs.values_list('pk',flat=True)),5)):
    print()
    print(d.title)
    print(d.content)
    


2509
title__iregex
content__iregex
7290 486
###
 all query docs

Assessing vulnerability to climate change and socioeconomic stressors in the Reef Islands group, Solomon Islands
Screening, evaluation, and ranking of oil reservoirs suitable for CO2-flood EOR and carbon dioxide sequestration
Gambling over Growth: Economic Uncertainty, Discounting, and Regulatory Policy
Growing "Smart'? Urbanization Processes in the Pune Urban Agglomeration
Safe operating space for humanity at a regional scale
###
 p > 0.2


Potential increase in floods in California's Sierra Nevada under future climate projections
California's mountainous topography, exposure to occasional heavily moisture-laden storm systems, and varied communities and infrastructures in low lying areas make it highly vulnerable to floods. An important question facing the state-in terms of protecting the public and formulating water management responses to climate change-is "how might future climate changes affect flood characteristics 

In [18]:
users = User.objects.filter(username__in=[
    #"michael.hegarty@climateanalytics.org",
    "emily.theokritoff@climateanalytics.org"
])
tag, created = Tag.objects.get_or_create(query=q,title="Infrastructure, settlements 2 - prediction > 0.2")
print(tag, created)
for i,d in enumerate(Doc.objects.filter(pk__in=random.sample(list(r_docs.values_list('pk',flat=True)),40))):
    u = users[0]
    d.tag.add(tag)
    do, created = DocOwnership.objects.get_or_create(
        doc=d,
        query=q,
        tag=tag,
        user=u
    ) 
tag.update_tag()

Infrastructure, settlements 2 - prediction > 0.2 True


In [88]:


django.db.connection.close()
assigned = set(DocOwnership.objects.filter(query__project=project).values_list('doc__id',flat=True))
print(len(assigned))

s = "land use, land management" 
inclusion = s.replace(', ','|')
se = "species, bird, mammal, fish, ecosystem"
exclusions = se.replace(', ','|')

p = 0.2

pds = predicted_ids[predicted_ids['prediction']>p]

dids = set([])
eids = set([])
for f in ["title__iregex","content__iregex"]:
    print(f)
    dids = dids | set(q.doc_set.filter(**{f: inclusion}).values_list('pk',flat=True))
    eids = eids | set(q.doc_set.filter(**{f: exclusions}).values_list('pk',flat=True))
    
doc_ids = dids - eids
    
docs = Doc.objects.filter(pk__in=doc_ids).exclude(id__in=assigned)

#docs = q.doc_set.filter(title__iregex=inclusion).exclude(id__in=assigned).exclude(content__iregex=exclusions)
r_docs = Doc.objects.filter(id__in=doc_ids & set(pds['id']))

print(docs.count(), r_docs.count())

print('###\n all query docs\n')

for d in Doc.objects.filter(pk__in=random.sample(list(docs.values_list('pk',flat=True)),5)):
    print(d.title)
    
print(f'###\n p > {p}\n')

for d in Doc.objects.filter(pk__in=random.sample(list(r_docs.values_list('pk',flat=True)),5)):
    print()
    print(d.title)
    print(d.content)
    


2214
title__iregex
content__iregex
9038 1352
###
 all query docs

Did irrigation impact 20th century air temperature in the High Plains aquifer region?
Evaluating adaptation and the production development of Finnish agriculture in climate and global change
Climate change impact, mitigation and adaptation strategies for agricultural and water resources, in Ganga Plain (India)
Accuracy assessment of land cover/land use classifiers in dry and humid areas of Iran
Returns on investment in watershed conservation: Application of a best practices analytical framework to the Rio Camboriu Water Producer program, Santa Catarina, Brazil
###
 p > 0.2


Scenario-Based Impact Assessment of Land Use/Cover and Climate Changes on Watershed Hydrology in Heihe River Basin of Northwest China
This study evaluated hydrological impacts of potential climate and land use changes in Heihe River Basin of Northwest China. The future climate data for the simulation with Soil and Water Assessment Tool (SWAT) were pr

In [89]:
users = User.objects.filter(username__in=[
    "michael.hegarty@climateanalytics.org",
    "emily.theokritoff@climateanalytics.org"
])
tag, created = Tag.objects.get_or_create(query=q,title="Land use - prediction>0.2")
print(tag, created)
for i,d in enumerate(Doc.objects.filter(pk__in=random.sample(list(r_docs.values_list('pk',flat=True)),40))):
    u = users[i%2]
    d.tag.add(tag)
    do, created = DocOwnership.objects.get_or_create(
        doc=d,
        query=q,
        tag=tag,
        user=u
    ) 
tag.update_tag()

Land use - prediction>0.2 True


# Teleconnections, extremes

In [75]:


django.db.connection.close()
assigned = set(DocOwnership.objects.filter(query__project=project).values_list('doc__id',flat=True))
print(len(assigned))

s = "temperature extreme, extreme temperature" 
inclusion = s.replace(', ','|')
se = "oscillation, sea surface temperature, SST, radiation"
exclusions = se.replace(', ','|')

p = 0.4

pds = predicted_ids[predicted_ids['prediction']>p]

dids = set([])
eids = set([])
for f in ["title__iregex","content__iregex"]:
    print(f)
    dids = dids | set(q.doc_set.filter(**{f: inclusion}).values_list('pk',flat=True))
    eids = eids | set(q.doc_set.filter(**{f: exclusions}).values_list('pk',flat=True))
    
doc_ids = dids - eids
    
docs = Doc.objects.filter(pk__in=doc_ids).exclude(id__in=assigned)

#docs = q.doc_set.filter(title__iregex=inclusion).exclude(id__in=assigned).exclude(content__iregex=exclusions)
r_docs = Doc.objects.filter(id__in=doc_ids & set(pds['id']))

print(docs.count(), r_docs.count())

print('###\n all query docs\n')

for d in Doc.objects.filter(pk__in=random.sample(list(docs.values_list('pk',flat=True)),5)):
    print(d.title)
    
print(f'###\n p > {p}\n')

for d in Doc.objects.filter(pk__in=random.sample(list(r_docs.values_list('pk',flat=True)),5)):
    print()
    print(d.title)
    


2753
title__iregex
content__iregex
1726 368
###
 all query docs

Heat-induced mortality and expression of heat shock proteins in Colorado potato beetles treated with imidacloprid
GREEN INFRASTRUCTURE CONTRIBUTION FOR CLIMATE CHANGE ADAPTATION IN URBAN LANDSCAPE CONTEXT
Detection of infectious bursal disease virus from formalin-fixed paraffin-embedded tissue by immunohistochemistry and real-time reverse transcription-polymerase chain reaction
Tolerance to environmental stress by the nitrogen-fixing actinobacterium Frankia and its role in actinorhizal plants adaptation
Impact of Extreme Heat on Corn Yield in Main Summer Corn Cultivating Area of China at Present and Under Future Climate Change
###
 p > 0.4


Changes in extreme temperatures of Australasian summer simulated by CCAM under global warming, and the roles of winds and land-sea contrasts

Changes in daily maximum temperature extremes across India over 1951-2014 and their relation with cereal crop productivity

Trend and Abrupt Re

In [76]:
users = User.objects.filter(username__in=[
    "kaylin.lee@climateanalytics.org"
])
tag, created = Tag.objects.get_or_create(query=q,title="temperature extremes - prediction>0.2")
print(tag, created)
for i,d in enumerate(Doc.objects.filter(pk__in=random.sample(list(r_docs.values_list('pk',flat=True)),40))):
    u = users[0]
    d.tag.add(tag)
    do, created = DocOwnership.objects.get_or_create(
        doc=d,
        query=q,
        tag=tag,
        user=u
    ) 
tag.update_tag()

temperature extremes - prediction>0.2 True


(11, {'scoping.DocOwnership': 11})

In [77]:


django.db.connection.close()
assigned = set(DocOwnership.objects.filter(query__project=project).values_list('doc__id',flat=True))
print(len(assigned))

s = "teleconnection, oscillation" 
inclusion = s.replace(', ','|')
se = "asfasdfasdf"
exclusions = se.replace(', ','|')

p = 0.4

pds = predicted_ids[predicted_ids['prediction']>p]

dids = set([])
eids = set([])
for f in ["title__iregex","content__iregex"]:
    print(f)
    dids = dids | set(q.doc_set.filter(**{f: inclusion}).values_list('pk',flat=True))
    eids = eids | set(q.doc_set.filter(**{f: exclusions}).values_list('pk',flat=True))
    
doc_ids = dids - eids
    
docs = Doc.objects.filter(pk__in=doc_ids).exclude(id__in=assigned)

#docs = q.doc_set.filter(title__iregex=inclusion).exclude(id__in=assigned).exclude(content__iregex=exclusions)
r_docs = Doc.objects.filter(id__in=doc_ids & set(pds['id']))

print(docs.count(), r_docs.count())

print('###\n all query docs\n')

for d in Doc.objects.filter(pk__in=random.sample(list(docs.values_list('pk',flat=True)),5)):
    print(d.title)
    
print(f'###\n p > {p}\n')

for d in Doc.objects.filter(pk__in=random.sample(list(r_docs.values_list('pk',flat=True)),5)):
    print()
    print(d.title)
    


2785
title__iregex
content__iregex
15737 1429
###
 all query docs

Interactions between the Atlantic Multidecadal Oscillation, El Nino/La Nina, and the PNA in winter Mississippi valley stream flow
Phylogeography and genetic structure of the red-legged partridge (Alectoris rufa): more evidence for refugia within the Iberian glacial refugium
Effects of precipitation and potential evaporation on actual evapotranspiration over the Laohahe basin, northern China
Interannual variability of nearbed sediment flux on the Eel River shelf, northern California
Quantification and Exploration of Diurnal Oscillations in Tropical Cyclones
###
 p > 0.4


Twentieth-century decline in streamflows from the hydrographic apex of North America

How ENSO affects maize yields in China: understanding the impact mechanisms using a process-based crop model

Using hierarchical models to estimate effects of ocean anomalies on north-west Pacific Chinook salmon Oncorhynchus tshawytscha recruitment

Detailed assessment

In [78]:
users = User.objects.filter(username__in=[
    "shruti.nath@climateanalytics.org"
])
tag, created = Tag.objects.get_or_create(query=q,title="oscillations, teleconnections - prediction>0.2 2")
print(tag, created)
tag.docownership_set.filter(relevant=0).delete()
tag.doc_set.clear()
for i,d in enumerate(Doc.objects.filter(pk__in=random.sample(list(r_docs.values_list('pk',flat=True)),40))):
    u = users[0]
    d.tag.add(tag)
    do, created = DocOwnership.objects.get_or_create(
        doc=d,
        query=q,
        tag=tag,
        user=u
    ) 
tag.update_tag()

oscillations, teleconnections - prediction>0.2 2 True


# Wind

In [80]:
tag = Tag.objects.get(query=q,title="wind speed, ET - prediction>0.2")
tag_dos = tag.docownership_set.values('doc__id')
tag_dors = tag.docownership_set.values('relevant').annotate(n=Count('pk'))
print(list(tag_dors.order_by('-n')))
ducs = DocUserCat.objects.filter(doc__id__in=tag_dos,category__level=4).values('category__name').annotate(n=Count('pk'))
list(ducs.order_by('-n'))

[{'relevant': 2, 'n': 14}, {'relevant': 0, 'n': 14}, {'relevant': 1, 'n': 12}]


[{'category__name': '02 Air or land surface temperature changes', 'n': 3},
 {'category__name': '25 Evapotranspiration', 'n': 3},
 {'category__name': '06 Humidity', 'n': 2},
 {'category__name': '52 Shifts in phenology (Terrestrial and freshwater)',
  'n': 2},
 {'category__name': '05 Changes in precipitation', 'n': 2},
 {'category__name': '09 Atmospheric/marine circulation or teleconnections',
  'n': 1},
 {'category__name': '26 Drought frequency and intensity', 'n': 1},
 {'category__name': '28 River runoff', 'n': 1},
 {'category__name': '08 Changes in strong precipitation', 'n': 1}]

In [50]:
django.db.connection.close()
assigned = set(DocOwnership.objects.filter(query__project=project).values_list('doc__id',flat=True))
print(len(assigned))

s = "wind speed, evapotranspiration, changes in precipitation, changes in strong precipitation, extreme temperature, soil moisture, humidity" 
s = "wind, storm, cyclone" 
inclusion = s.replace(', ','|')
se = "species, rivers, plants, lakes"
exclusions = se.replace(', ','|')

p = 0.3

pds = predicted_ids[predicted_ids['prediction']>p]

dids = set([])
eids = set([])
for f in ["title__iregex","content__iregex"]:
    print(f)
    dids = dids | set(q.doc_set.filter(**{f: inclusion}).values_list('pk',flat=True))
    eids = eids | set(q.doc_set.filter(**{f: exclusions}).values_list('pk',flat=True))
    
doc_ids = dids - eids
    
docs = Doc.objects.filter(pk__in=doc_ids).exclude(id__in=assigned)

#docs = q.doc_set.filter(title__iregex=inclusion).exclude(id__in=assigned).exclude(content__iregex=exclusions)
r_docs = Doc.objects.filter(id__in=doc_ids & set(pds['id']))

print(docs.count(), r_docs.count())

print('###\n all query docs\n')

for d in Doc.objects.filter(pk__in=random.sample(list(docs.values_list('pk',flat=True)),5)):
    print(d.title)
    
print(f'###\n p > {p}\n')

for d in Doc.objects.filter(pk__in=random.sample(list(r_docs.values_list('pk',flat=True)),5)):
    print()
    print(d.title)
    


2931
title__iregex
content__iregex
36727 2457
###
 all query docs

Characterization of an island aquifer from tidal response
Thinking globally, acting locally - institutionalizing climate change at the local government level in Durban, South Africa
Impacts of Natural Disasters on Swedish Electric Power Policy: A Case Study
THE CORRUBEDO BEACH LAGOON COMPLEX, GALICIA, SPAIN - DYNAMICS, SEDIMENTS AND RECENT EVOLUTION OF A MESOTIDAL COASTAL EMBAYMENT
Novel Metrics for Evaluation of Ensemble Forecasts of Tropical Cyclone Structure
###
 p > 0.3


On climatic fluctuations and environmental changes of the Indo-Gangetic Plains, India

Detection of trends in annual extreme rainfall

Temporal fluctuations in weather and climate extremes that cause economic and human health impacts: A review

The tropical ocean response to a change in solar forcing

PROVISIONALLY CORRECTED SURFACE WIND DATA, WORLDWIDE OCEAN ATMOSPHERE SURFACE FIELDS, AND SAHELIAN RAINFALL VARIABILITY


In [51]:
users = User.objects.filter(username__in=[
    "peter.pfleiderer@climateanalytics.org"
])
tag, created = Tag.objects.get_or_create(query=q,title="wind speed, ET - prediction>0.2")
print(tag, created)
for i,d in enumerate(Doc.objects.filter(pk__in=random.sample(list(r_docs.values_list('pk',flat=True)),40))):
    u = users[0]
    d.tag.add(tag)
    do, created = DocOwnership.objects.get_or_create(
        doc=d,
        query=q,
        tag=tag,
        user=u
    ) 
tag.update_tag()

wind speed, ET - prediction>0.2 False
